In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections
import pickle

import copy 
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import models

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from operator import add

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
GOD = pickle.load(open('GOD.pkl','rb'))
D = pickle.load(open('COUNT','rb'))
df_sample_test = pickle.load(open('df_sample_test','rb'))
labels_sample = df_sample_test['digit'].to_numpy()
df_sample_test = df_sample_test.drop(['digit'], axis=1)

In [5]:
from tqdm.notebook import tqdm
HISTORY =[]
for rounds in tqdm(range(1)):
    
    def getModel():
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(10,activation="sigmoid")
        ])
        model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
        return model
    
    import numpy as np
    from collections import Counter
    

    def getweight(df):
        LBL = Counter(df.digit)
        maen_arr = np.mean(list(map(lambda x:x[1]/D[x[0]],list(LBL.items()))))
        return maen_arr
    
    def train_on_clients():
        for writer in range(1,31):
            df = FROUND.get(writer)
#             cw.append(getweight(df))
            labels = df['digit'].to_numpy()
            df_copy = df.drop(['digit','writer'], axis=1)
            model = getModel()
            model.fit(df_copy.to_numpy(), labels, epochs=10,verbose=0)
            h = model.evaluate(df_sample_test.to_numpy(), labels_sample,verbose=0)
            cw.append(h[1])
            writer_models.append(model)
    
    def average(rounds, weights):
    
        agg_model = rounds[0]
        l1,b1,l2,b2 = agg_model.get_weights()
        w1 = weights[0]

        l1*=w1
        b1*=w1
        l2*=w1
        b2*=w1

#         weights = np.cbrt(weights)

        CONST = sum(weights)

        for i,w in zip(rounds[1:],weights[1:]):
            tl1,tb1,tl2,tb2 = i.get_weights()
            l1+=(tl1*w)
            b1+=(tb1*w)        
            l2+=(tl2*w)        
            b2+=(tb2*w)

        agg_model.set_weights([l1/CONST,b1/CONST,l2/CONST,b2/CONST])
        return agg_model
    
    FROUND = GOD[rounds].get('fs')
    OROUND = GOD[rounds].get('snd')
    
    writer_models = []
    cw = []

    train_on_clients()
    
    DF_TEST = pd.read_csv('TEST.csv').iloc[:,1:]
    labels_tes = DF_TEST['digit'].to_numpy()
    df_tes = DF_TEST.drop(['digit'], axis=1)
    
    GLOBAL = []
    history = []
    
    agg_model = average(writer_models,cw)
        
    history.append(agg_model.evaluate(df_tes.to_numpy(), labels_tes,verbose=0))
    
    for i in range(10):
        rounds = []
        cw_local = []
        for writer in range(1,31):
            df = OROUND[i].get(writer)
#             cw_local.append(getweight(df))
            labels = df['digit'].to_numpy()
            df_copy = df.drop(['digit','writer'], axis=1)
            model = agg_model
            HIS = model.fit(df_copy.to_numpy(), labels, epochs=10,verbose=0)
#             cw_local.append(HIS.history['accuracy'][-1])
            rounds.append(model)
            
#             print("sample test")
            h = model.evaluate(df_sample_test.to_numpy(), labels_sample,verbose=0)
            cw_local.append(h[1])
            
        agg_model = average(rounds,cw_local)
        GLOBAL.append({
            'round {}'.format(i):agg_model
        })
#         print('----------Global Model-----------')
        his = agg_model.evaluate(df_tes.to_numpy(), labels_tes, verbose=0)
        history.append(his)
    HISTORY.append(history)
pickle.dump(HISTORY,open('agg6','wb'))